In [260]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import os
from itertools import chain
import glob
import seaborn as sns
import matplotlib.font_manager as font_manager
import seaborn as sns
import matplotlib as mpl
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter
from sklearn.model_selection import GridSearchCV
from pandas import read_csv
from datetime import datetime
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import register_matplotlib_converters
import warnings
warnings.filterwarnings('ignore')
register_matplotlib_converters()
sns.set_theme()
mpl.matplotlib_fname()
font_dirs = ['D:\\Software\\Anaconda\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf' ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
mpl.font_manager._rebuild()
result = font_manager.findfont("Rounded Mplus 1c")
print(result)
mpl.rc('font', family='Rounded Mplus 1c')


le = LabelEncoder()
ss = StandardScaler()

D:\Software\Anaconda\Lib\site-packages\matplotlib\mpl-data\fonts\ttf\MPLUSRounded1c-Regular.ttf


In [36]:
names = ["Gradient_Boosting", "Decision_Tree", "Extra_Trees", "Random_Forest","XGBoost","LGBMRegressor"]
models = [
    GradientBoostingRegressor(),
    DecisionTreeRegressor(max_depth=5),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    XGBRegressor(),
    LGBMRegressor()]

In [3]:
df = pd.read_csv('Kaiho_merged_data.csv')

In [6]:
df.columns

Index(['Vị trí_ロケーション', 'Ngày sản xuất_生産日', 'Tháng sản xuất_生産月',
       'Năm sản xuất_生産年', 'Nv sản xuất_生産担当者', 'Số nhập kho_入庫番号',
       'Nguồn nhập hàng_仕入先', 'Mã loại phụ tùng_商品コード', 'Tên phụ tùng_商品名',
       'Số nhập kho(phụ tùng)_入庫番号 (部品)', 'Tên xe_車名', 'Model xe_車輌型式',
       'Engine model_エンジン型式', 'Số lượng_数量_PARTS', 'Tổng giá nhập_仕入合計',
       'Số phút công_標準工数', 'Kết quả_生産高',
       'Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額',
       'Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額',
       'Số lượng bán ra_販売数量', 'Thành tích bán ra_販売実績', 'Khách hàng_仕向先',
       'Truyền động 1_駆動1', 'TM_TM', 'Truyền động 2_駆動2', 'Chi tiết_詳細',
       'Số km đã chạy_走行距離', 'Năm sản xuất_年式', 'Tình trạng engine_エンジン状態',
       'Rank_ランク', 'Loại xe_車種', 'Tình trạng sản xuất_未生産',
       'Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額.1',
       'Số chỉ định loại_型式指定番号', 'Số phân loại danh mục_類別区分番号',
       'Đơn giá mua_仕入単価', 'Nhà sản xuất_メーカー', 'Giờ làm việc

In [19]:
df['date_time']  = df['Năm bán hàng_売上年'].astype(str) +'-'+df['Tháng bán hàng_売上月'].astype(str) 

In [20]:
df_copy = df.copy()

In [21]:
df_copy

,Vị trí_ロケーション,Ngày sản xuất_生産日,Tháng sản xuất_生産月,Năm sản xuất_生産年,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),...,Mã nhà cung cấp_仕入先コード,Số body_車体番号,CONTAINER_CONTAINER,Tên khách hàng_売上先名,Phân loại mang đến/đón_持込・引取区分,Tiền công_工賃,D/G_Ｄ／Ｇ,Phân loại bán hàng_売上分類,Đang tải nguồn_積込元,date_time
0,会宝産業本社,17,11,2015,山下大輝,'0000000302676,西田 敏勝,'110010,ノーズカット（個別）,'0000007988203,...,990010.0,'1001019,NaN,"一番オート OOO ""LIGHT VL""",NaN,0.0,G,NaN,会宝産業本社,2016-3
1,会宝産業本社,3,2,2015,中嶋 修,'0000000303771,ｴｻﾞｼ自動車,'100000,エンジン,'0000007739581,...,990000.0,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,NaN,0.0,G,NaN,会宝産業本社,2015-2
2,会宝産業本社,3,2,2015,楊 志毅,'0000000303771,ｴｻﾞｼ自動車,'110010,ノーズカット（個別）,'0000007712751,...,990000.0,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,NaN,0.0,G,NaN,会宝産業本社,2015-2
3,会宝産業本社,5,2,2015,西川航平,'0000000314958,ＪＵ石川,'110010,ノーズカット（個別）,'0000007732276,...,319800.0,'2010272,40HC,㈱パシフィック・トレーディング・インターナショナル,NaN,0.0,G,NaN,会宝産業本社,2015-2
4,会宝産業本社,25,2,2015,中里太明,'0000000319165,会宝パーツサービス㈲福井店 アップガレージ,'110010,ノーズカット（個別）,'0000007742703,...,208750.0,'072307,40HC,㈱パシフィック・トレーディング・インターナショナル,NaN,0.0,G,NaN,会宝産業本社,2015-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20423,会宝産業本社,10,9,2012,中谷祐騎,'0000000297532,石川トヨタ本社,'110010,ノーズカット（個別）,'0000007022141,...,111300.0,'0113274,20,"TTO Co., LTD",NaN,0.0,G,NaN,会宝産業本社,2012-9
20424,会宝産業本社,27,8,2012,坂尻僚平,'0000000297920,ＪＵ石川,'140092,Ｆアクスル（生/Ｄ）,'0000007052516,...,319800.0,'0020874,20,FADEEV EVGENY,NaN,0.0,D,NaN,会宝産業本社,2012-9
20425,会宝産業本社,23,8,2012,菊池忠義,'0000000298205,西村 信與,'140092,Ｆアクスル（生/Ｄ）,'0000005002723,...,990010.0,'101069,20,FADEEV EVGENY,NaN,0.0,D,NaN,会宝産業本社,2012-9
20426,会宝産業本社,6,11,2012,坂尻僚平,'0000000300843,エステート,'140153,Ｒアクスル（生/Ｄ）,'0000007076751,...,135300.0,'0010109,20,FADEEV EVGENY,NaN,0.0,D,NaN,会宝産業本社,2012-11


In [268]:
scale = le.fit_transform(df_copy['date_time'])

In [269]:
scale

array([87, 74, 74, ..., 45, 36, 37])

In [262]:
df_copy.shape

(20428, 60)

In [263]:
data.shape

(3, 2)

In [385]:
list_feature = [
                'Tên phụ tùng_商品名',
#                 'Tên xe_車名',
                'Model xe_車輌型式',
                'Engine model_エンジン型式',
                'Thành tích bán ra_販売実績'
#                 'date_time',
#                'D/G_Ｄ／Ｇ',
#                'Loại xe_車種'
               ]

In [386]:
# df_group_month = df_copy.groupby(list_feature).agg(revenue = ('Thành tích bán ra_販売実績','sum')).reset_index()
df_group_month = df_copy[list_feature]

In [387]:
# df_group_month = df_group_month.sort_values(by=['date_time'])
# df_group_month

In [388]:
for col in df_group_month.columns[df_group_month.dtypes == object]:
    print(col, df_group_month[col].nunique())
    df_group_month[col] = df_group_month[col].fillna("unknown")
    df_group_month[col] = le.fit_transform(df_group_month[col].values)

for col in df_group_month.columns[df_group_month.dtypes == 'float64']:
    df_group_month.fillna(df_group_month[col].mean(), inplace=True)

Tên phụ tùng_商品名 12
Model xe_車輌型式 1587
Engine model_エンジン型式 536


In [389]:
# df_group_month.date_time.unique()

# Without Scaler

In [390]:
target = df_group_month['Thành tích bán ra_販売実績']

In [391]:
data = df_group_month.drop(columns=['Thành tích bán ra_販売実績'])

In [392]:
X_train, X_test, y_train, y_test =  train_test_split(data, target, test_size= 0.3, random_state=42)

In [393]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
#     print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test - y_pred)/y_test)*100)
    print(mape_score)
#     mape_score = mean_absolute_percentage_error(y_test,y_pred)
    r_squared_score  = model.score(X_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

30.063388062000868
29.51835883152997
16.87972000195691
16.782006570232518
20.716877942046718
21.445891887174398


In [134]:
(y_test - y_pred).shape

(5029,)

# With Scaler

In [394]:
df_scale = pd.DataFrame(ss.fit_transform(df_group_month),columns=df_group_month.columns)

In [395]:
target_scaler = df_scale['Thành tích bán ra_販売実績']

In [396]:
target_scaler

0       -0.697563
1       -0.200797
2       -0.752760
3       -0.697563
4       -0.752760
           ...   
20423   -0.697563
20424   -0.932148
20425   -0.932148
20426   -0.890750
20427   -0.890750
Name: Thành tích bán ra_販売実績, Length: 20428, dtype: float64

In [397]:
data = df_scale.drop(columns=['Thành tích bán ra_販売実績'])

In [398]:
X_train, X_test, y_train, y_test =  train_test_split(data, target, test_size= 0.3, random_state=42)

In [399]:
y_test

11876     35000.0
6735      20000.0
12902     30000.0
19893     42000.0
20269     10000.0
           ...   
17288     30000.0
16615     25000.0
7638     100000.0
8361     180000.0
15451      8000.0
Name: Thành tích bán ra_販売実績, Length: 6129, dtype: float64

In [400]:
y_pred

array([ 37889.1083147 ,  23307.18289944,  37737.90405802, ...,
       139063.31555807, 126280.83128107,   8815.57666608])

In [401]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
#     print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test - y_pred)/y_test)*100)
    print(mape_score)
#     mape_score = mean_absolute_percentage_error(y_test,y_pred)
    r_squared_score  = model.score(X_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

30.013371487088097
29.51217812572254
16.887852362428855
16.739483375273572
20.73886563880307
21.627152411092542


In [218]:
(y_test - y_pred).shape

(5329,)

In [219]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.823,18027.637,20.139
1,Decision_Tree,0.814,18492.234,20.685
2,Extra_Trees,0.741,21805.979,20.183
3,Random_Forest,0.780,20088.236,20.402
4,XGBoost,0.797,19332.064,21.030
5,LGBMRegressor,0.819,18249.618,19.388


In [220]:
y_pred

array([87565.98177763, 90488.39485411, 83054.88794164, ...,
       28129.22654704, 10445.3873728 , 44312.15762412])

In [221]:
target_scaled = ss.fit_transform(df_group_month[['revenue']])
l = ss.inverse_transform(y_pred)
l

array([3.77504654e+09, 3.90103291e+09, 3.58057152e+09, ...,
       1.21270480e+09, 4.50347589e+08, 1.91035729e+09])

In [222]:
k = ss.inverse_transform(y_test)
k

array([3.87997802e+09, 3.87997802e+09, 3.23332216e+09, ...,
       1.07780263e+09, 4.31146765e+08, 1.72445849e+09])

In [223]:
mean_squared_error(l,k,squared= False)

786748157.4494454

In [224]:
model  = XGBRegressor()

In [225]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [226]:
X_test

,Tên phụ tùng_商品名,Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,Đơn giá bán hàng_売上単価,date_time
15277,-0.410486,-1.056367,-1.102525,-1.105632,1.370901,1.223949
6444,2.703361,-1.160204,0.388760,-1.175921,1.370901,-0.506360
10085,-0.410486,-1.104292,1.318080,-0.837257,0.974679,0.223614
11011,-0.410486,-0.121832,-0.109793,-1.073682,-0.213985,0.412867
5155,-0.410486,-1.855115,0.119804,0.562129,-0.213985,-0.695612
...,...,...,...,...,...,...
6186,-0.410486,-1.024417,-1.323375,0.376823,-0.213985,-0.533396
12400,-0.410486,0.621004,-0.934154,2.095703,-0.213985,0.656191
14071,-0.410486,0.405342,0.570251,1.533393,-0.346059,0.980624
8395,-0.021255,-0.145795,1.759780,0.798555,-0.742280,-0.181927


In [227]:
y_pred = model.predict(X_test)

In [228]:
mean_absolute_percentage_error(y_test,y_pred)

0.2102952441410314

In [259]:
test = ['エンジン','ﾗﾌｪｽﾀ','NB30','MR20DE','40000.0','2019-9']
encode = le.fit_transform(np.array([[1, 2], [3, 1], [3, 2]]))
test_encode = encode.transform(test)

ValueError: y should be a 1d array, got an array of shape (3, 2) instead.

In [257]:
test_encode

array([4, 5, 3, 2, 1, 0])

In [251]:
model.predict(np.array(test_encode).reshape(1,len(test_encode)))

TypeError: object of type 'LabelEncoder' has no len()

In [245]:
from sklearn.preprocessing import StandardScaler
data = np.array([[1, 2], [3, 1], [3, 2]])
scaler = StandardScaler()
print(scaler.fit_transform(data))

[[-1.41421356  0.70710678]
 [ 0.70710678 -1.41421356]
 [ 0.70710678  0.70710678]]


In [246]:
data.shape

(3, 2)